# Transfer Entropy

In [1]:
import numpy as np
import nolds
import scipy.io
import mne
import pandas as pd
import seaborn as sns
import pyqtgraph as pg
from scipy.io import loadmat
from scipy import stats
import matplotlib.pyplot as plt
import antropy as ent
from attractors import Attractor
from antropy import higuchi_fd
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.computation import RQAComputation
from pyrqa.time_series import TimeSeries
from pyrqa.result import RQAResult
from pyrqa.opencl import OpenCL
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.image_generator import ImageGenerator
import numpy as np
import matplotlib.pyplot as plt
from antropy import higuchi_fd

In [2]:
# Load data
data = loadmat('/home/vincent/AAA_projects/University/MSc_AI/Final_Project/downsampled/EEG_DS_Struct_0101.mat')

In [3]:
DSamp = data['DSamp']

triggers = DSamp[0][0][0]
EEGdata = DSamp[0][0][1]
fs = DSamp[0][0][2][0][0] 
fsOld = DSamp[0][0][3][0][0] 
time = DSamp[0][0][4][0]
label = DSamp[0][0][5]
nchan = DSamp[0][0][6][0][0]
rate = DSamp[0][0][7][0][0]
npt = DSamp[0][0][8][0][0]
Subj = DSamp[0][0][9][0]
ptrackerPerf = DSamp[0][0][10]
ptrackerTime = DSamp[0][0][11]
ptrackerfs = DSamp[0][0][12][0][0]

In [4]:
def phase_randomize(ts):
    ts_randomized = np.empty_like(ts)
    for i in range(ts.shape[0]):
        # Fourier transform
        ts_fourier = np.fft.rfft(ts[i])

        # Generate random phases
        random_phases = np.exp(1j * np.random.uniform(0, 2*np.pi, len(ts[i]) // 2 + 1))

        # Apply the random phases to the Fourier transform
        ts_fourier_randomized = ts_fourier * random_phases

        # Inverse Fourier transform
        ts_randomized[i] = np.fft.irfft(ts_fourier_randomized, len(ts[i]))
        
    return ts_randomized

In [5]:
import numpy as np
from pyinform import transfer_entropy
import matplotlib.pyplot as plt

# Define TES stimulation parameters
stimulation_params = {
    "frequency": 10,  # Hz
    "amplitude": 1.5,  # mA
    "duration": 5  # minutes
}

# Apply TES stimulation
stimulation_duration = stimulation_params["duration"] * 60  # Convert to seconds
stimulation_frequency = stimulation_params["frequency"]
stimulation_amplitude = stimulation_params["amplitude"]
stimulation_waveform = stimulation_amplitude * np.sin(2 * np.pi * stimulation_frequency * np.arange(stimulation_duration))

# Define time delay and embedding dimension for TE analysis
tau = 10
m = 3

# Generate surrogate data for TE analysis
surrogate_EEGdata = phase_randomize(EEGdata)  # Corrected here
surrogate_stimulation_waveform = phase_randomize(stimulation_waveform)

# Calculate transfer entropy from TES to EEG
te_stim_to_eeg = transfer_entropy(surrogate_stimulation_waveform, EEGdata, k=m, l=tau)  # Corrected here


# Visualize the results
plt.subplot(2, 2, 1)
plt.plot(EEGdata)
plt.title("EEG data")

plt.subplot(2, 2, 2)
plt.plot(stimulation_waveform)
plt.title("TES stimulation")

plt.subplot(2, 2, 3)
plt.plot(EEGdata + stimulation_waveform)
plt.title("EEG data with TES stimulation")

plt.subplot(2, 2, 4)
plt.imshow(te_stim_to_eeg, cmap='jet', origin='lower', aspect='auto')
plt.title("Transfer entropy (TES to EEG)")
plt.xlabel("TES signal")
plt.ylabel("EEG signal")

plt.tight_layout()
plt.show()

IndexError: tuple index out of range